# Objective

You need to start by reading the [scrape_search notebook](https://github.com/morschulik/scrape_amazon_reviews/blob/main/Jupyter_notebook/scrap_search.ipynb)

Then the [scrape reviews](https://github.com/morschulik/scrape_amazon_reviews/blob/main/Jupyter_notebook/scrap_reviews.ipynb)


In the main.py we want to combine both of them and create a function which retrieve all the products from a search query with its reviews and parse the results.


This function will receive a search query(keyword) and return end result list combining the keyword, the title of the product the review details etc.

In the automated scraper we have a trial to automatically run the code weekly, compares the recent results with the current ones and add write the results in a new file if there is a difference. 
Its aim is to keep a history of the data. The output could not be examined yet.

# The main.py
We need to import the functions from the scrap_search and scrape_reviews(make sure that those files are in the same directory like your notebook)

In [8]:
from scraper.scrap_search import search
from scraper.scrap_reviews import *

the combining function will like following

In [10]:
async def get_reviews_by_keyword(keyword):
    query = keyword.replace(' ', '+')
    limits = httpx.Limits(max_connections=5)
    async with httpx.AsyncClient(limits=limits, timeout=httpx.Timeout(20.0), headers=BASE_HEADERS) as session:

        products_list = await search(query, session=session)
        # asins_list = [product["asin"] for product in products_list]
        end_results_list = []
        for product in products_list:
            product_reviews = await scrape_reviews(product["asin"], session=session)
            for review in product_reviews:
                end_results = {
                    "search keyword": keyword,
                    "Product_asin": product["asin"],
                    "Product_title": product["title"],
                    "Product_url": product["url"],
                    "review_title": review["title"],
                    "review_rating": review["star_rating"],
                    "review_verified": review["verified"],
                    "review_location": review["location"],
                    "review_date": review["date"],
                    "review_body": review["body"],
                }
                end_results_list.append(end_results)
    return end_results_list


In [ ]:
# this code will take a while!
search_keyword = 'Refrigerator'
search_query = search_keyword.replace(' ', '+')
reviews_by_keyword = await get_reviews_by_keyword(search_query)

In [ ]:
print(reviews_by_keyword)

# Write the data to JSON or pandas data frame

In [13]:
i = int(input("Enter the file number four the output: "))

# dump the data to json file
with open(f'combined_results_{i}.json', 'w') as file:
    json.dump(reviews_by_keyword, file, indent=2)
# write the data to Excel file
df = pd.DataFrame(reviews_by_keyword,
                    columns=["search keyword", "Product_asin", "Product_title", "Product_url", "review_title",
                            "review_rating", "review_verified", "review_location", "review_date", "review_body", ])
df.to_excel(f"combined_results_{i}.xlsx", index=False)

# The automated scraper
Under examination. Still need to be checked better